##### ❗Προσοχή: Συμπληρώστε τα ονόματα των μελών της ομάδας σας

     Ονοματεπώνυμο 1ου μέλους:
             email 1ου μέλους:
  
     Ονοματεπώνυμο 2ου μέλους: Μαρία Ελπίδα Ντάφλου
             email 2ου μέλους: maria.elpida.nt@gmail.com, el21627@mail.ntua.gr


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.listdir('/content/drive/MyDrive/AI_24')

Mounted at /content/drive


['AI_24_EX2.ipynb',
 'PrologIntro.ipynb',
 'train_ratings.csv',
 'AI_24_ΕΧ1.ipynb',
 'db.pl',
 'test_ratings.csv',
 'movies_metadata.csv',
 'recommendations.pl',
 'AI_EX2_NEW.ipynb']

In [ ]:
movies_filename = '/content/drive/MyDrive/AI_24/movies_metadata.csv'

In [ ]:
%%capture
#install swi-prolog
!sudo apt-get install software-properties-common
!sudo apt-add-repository ppa:swi-prolog
!sudo apt-get update
!sudo apt-get install swi-prolog
#install pyswip
!pip install pyswip

##ΜΕΡΟΣ 1

In [ ]:
#prolog.assertz('(directed_by(X,Y) :- findall(M,director(M,X),Y))')

In [ ]:
import pandas as pd
from pyswip import Prolog
# Η βιβλιοθήκη pandas είναι χρήσιμη για την εργασία με τέτοια δεδομένα
import pandas as pd
path = '/content/drive/MyDrive/AI_24/'
# Διάβασμα του αρχείου 'movie_metadata.csv'
data = pd.read_csv(path + "movies_metadata.csv")
#Στο csv υπαρχούν κελία με nan τιμές
#Στις θέσεις αυτές βάζουμε 'UNK' πράγμα που  κάνουμε με την παρακάτω συνάρτηση
data.fillna("UNK", inplace=True)
# Preview the first 5 lines of the loaded data
data.head()

,Unnamed: 0,budget,genres,homepage,id,plot_keywords,language,original_title,overview,popularity,...,tagline,movie_title,vote_average,num_voted_users,title_year,country,director_name,actor_1_name,actor_2_name,actor_3_name
0,0,237000000,Action|Adventure|Fantasy|Science Fiction,http://www.avatarmovie.com/,19995,culture clash|future|space war|space colony|so...,English,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,Enter the World of Pandora.,Avatar,7.2,11800,2009.0,United States of America,James Cameron,Zoe Saldana,Sigourney Weaver,Stephen Lang
1,1,300000000,Adventure|Fantasy|Action,http://disney.go.com/disneypictures/pirates/,285,ocean|drug abuse|exotic island|east india trad...,English,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,2007.0,United States of America,Gore Verbinski,Orlando Bloom,Keira Knightley,Stellan Skarsgård
2,2,245000000,Action|Adventure|Crime,http://www.sonypictures.com/movies/spectre/,206647,spy|based on novel|secret agent|sequel|mi6|bri...,Français,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,A Plan No One Escapes,Spectre,6.3,4466,2015.0,United Kingdom,Sam Mendes,Christoph Waltz,Léa Seydoux,Ralph Fiennes
3,3,250000000,Action|Crime|Drama|Thriller,http://www.thedarkknightrises.com/,49026,dc comics|crime fighter|terrorist|secret ident...,English,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,The Legend Ends,The Dark Knight Rises,7.6,9106,2012.0,United States of America,Christopher Nolan,Michael Caine,Gary Oldman,Anne Hathaway
4,4,260000000,Action|Adventure|Science Fiction,http://movies.disney.com/john-carter,49529,based on novel|mars|medallion|space travel|pri...,English,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,"Lost in our world, found in another.",John Carter,6.1,2124,2012.0,United States of America,Andrew Stanton,Lynn Collins,Samantha Morton,Willem Dafoe


In [ ]:
def clean_text(text):
    text = text.replace(u'\xa0', u'')
    text = text.replace(u"'", u'')
    text = text.lstrip('\n')
    text = text.lstrip('"')
    text = text.rstrip('\n')
    text = text.rstrip(';')
    text = text.rstrip('')
    text = text.replace('""', '"')
    text = text.replace(';"', '')
    text = text.replace('";', '')
    text = text.replace('&amp"', '&')
    text = text.replace('"', '')
    text = text.replace("'", '')
    return text

In [ ]:
from pyswip import Prolog

prolog = Prolog()
literals = []
#create World
#Για κάθε row του πίνακα φτιάχνουμε τα κατηγορήματα που θέλουμε να αποθηκέυσουμε
#αρχικά σε μια λίστα με το όνομα literals

for row in data.itertuples(index=True, name='Pandas'):
    movie_title = clean_text(getattr(row, 'movie_title'))
    #country = clean_text(getattr(row, 'production_countries'))
    #keyword = clean_text(getattr(row, 'plot_keywords'))

    for genre in getattr(row, 'genres').split("|"):
        literals.append("genre('"+ movie_title +"','"+ genre +"')")

    #for country in getattr(row, 'production_countries').split("|"):
     #   literals.append("country('"+ movie_title +"','"+ country +"')")

    for keyword in getattr(row, 'plot_keywords').split("|"):
        literals.append("keyword('"+ movie_title +"','"+ keyword +"')")

    #prolog.assertz("has_country('" + movie_title + "','" + country.lower() + "')")

    #etc

    #prolog.assertz("has_country('" + movie_title + "','" + country.lower() + "')")
    #prolog.assertz("has_director('" + movie_title + "','" + getattr(row, 'director_name').lower() + "')")
    #prolog.assertz("has_color('" + movie_title + "','" + getattr(row, 'color').lower() + "')")
    #prolog.assertz("has_language('" + movie_title + "','" + getattr(row, 'language').lower() + "')")
    #prolog.assertz("cast('" + movie_title + "',[" + getattr(row, 'actor_1_name').lower() + "," + getattr(row, 'actor_2_name').lower() + "," + getattr(row, 'actor_3_name').lower() + "])")

#Η Prolog θέλει τα κατηγορήματά της με την σειρά
literals.sort()
for literal in literals:
  prolog.assertz(literal)
  print (literal +'.')

#Κανόνες:
prolog.assertz('similargenre(Movie1, Genre) :- genre(Movie1, Genre), genre(Movie2, Genre), Movie1 \= Movie2')
prolog.assertz('similarTheme(Movie1, Movie2) :- genre(Movie1, Genre), genre(Movie2, Genre), Movie1 \= Movie2, commonWordsInGenres(Movie1, Movie2, 5)')  # Προσαρμόστε το 5 ανάλογα με το πόσες λέξεις θέλετε να είναι κοινές
prolog.assertz('somewhatSimilarTheme(Movie1, Movie2) :- genre(Movie1, Genre), genre(Movie2, Genre), Movie1 \= Movie2, commonWordsInGenres(Movie1, Movie2, 3)')  # Προσαρμόστε το 3 ανάλογα με το πόσες λέξεις θέλετε να είναι κοινές
prolog.assertz('sameDirector(Movie1, Movie2) :- has_director(Movie1, Director), has_director(Movie2, Director), Movie1 \= Movie2')
prolog.assertz('sameColor(Movie1, Movie2) :- has_color(Movie1, Color1), has_color(Movie2, Color2), Color1 = Color2, Movie1 \= Movie2')
prolog.assertz('sameLanguage(Movie1, Movie2) :- has_language(Movie1, Language), has_language(Movie2, Language), Movie1 \= Movie2')
prolog.assertz('somewhatSameActors(Movie1, Movie2) :- cast(Movie1, Actors1), cast(Movie2, Actors2), intersection(Actors1, Actors2, Common), length(Common, 2)')


Streaming output truncated to the last 5000 lines.
genre('Roll Bounce','Drama').
genre('Roll Bounce','Family').
genre('Rollerball','Action').
genre('Rollerball','Science Fiction').
genre('Rollerball','Thriller').
genre('Romance & Cigarettes','Comedy').
genre('Romance & Cigarettes','Music').
genre('Romance & Cigarettes','Romance').
genre('Romeo + Juliet','Drama').
genre('Romeo + Juliet','Romance').
genre('Romeo Is Bleeding','Action').
genre('Romeo Is Bleeding','Crime').
genre('Romeo Is Bleeding','Drama').
genre('Romeo Is Bleeding','Thriller').
genre('Romeo Must Die','Action').
genre('Romeo Must Die','Crime').
genre('Romeo Must Die','Thriller').
genre('Ronin','Action').
genre('Ronin','Adventure').
genre('Ronin','Crime').
genre('Ronin','Thriller').
genre('Room','Drama').
genre('Room','Thriller').
genre('Rosemarys Baby','Drama').
genre('Rosemarys Baby','Horror').
genre('Rosemarys Baby','Mystery').
genre('Rosewater','Drama').
genre('Rotor DR1','Family').
genre('Rotor DR1','Science Fiction')

PrologError: Caused by: 'assertz((keyword('102 Dalmatians','women's prison'))).'. Returned: 'error(syntax_error(end_of_file_in_quoted(')), string(b'a', 0))'.

In [ ]:
from pyswip import Prolog

prolog = Prolog()

#create World
#Για κάθε row του πίνακα φτιάχνουμε τα κατηγορήματα που θέλουμε να αποθηκέυσουμε
#αρχικά σε μια λίστα με το όνομα literals

literals = []
movie_score = {}

  #for row in data.itertuples(index=True, name='Pandas'):
    #movie_title = clean_text(getattr(row, 'movie_title'))

  ############################

for genre in getattr(row, 'genres').split("|"):
    literals.append("genre('"+ movie_title +"','"+ genre +"')")

for row in data.itertuples(index=True, name='Pandas'):
    movie_title = clean_text(getattr(row, 'movie_title'))
    country = clean_text(getattr(row, 'country'))
    prolog.assertz("has_country('" + movie_title + "','" + country.lower() + "')")

  ############################

#Η Prolog θέλει τα κατηγορήματά της με την σειρά
literals.sort()
for literal in literals:
  prolog.assertz(literal)
  print (literal +'.')

#Επίσης μπορούμε να κάνουμε consult ένα έτοιμο αρχείο στον κόσμο όπως παρακάτω
#prolog.consult(path + "db.pl")


#Κανόνες:
prolog.assertz('similargenre(Movie1, Genre) :- genre(Movie1, Genre), genre(Movie2, Genre), Movie1 \= Movie2')
prolog.assertz('similarTheme(Movie1, Movie2) :- genre(Movie1, Genre), genre(Movie2, Genre), Movie1 \= Movie2, commonWordsInGenres(Movie1, Movie2, 5)')  # Προσαρμόστε το 5 ανάλογα με το πόσες λέξεις θέλετε να είναι κοινές
prolog.assertz('somewhatSimilarTheme(Movie1, Movie2) :- genre(Movie1, Genre), genre(Movie2, Genre), Movie1 \= Movie2, commonWordsInGenres(Movie1, Movie2, 3)')  # Προσαρμόστε το 3 ανάλογα με το πόσες λέξεις θέλετε να είναι κοινές
prolog.assertz('sameDirector(Movie1, Movie2) :- director(Movie1, Director), director(Movie2, Director), Movie1 \= Movie2')
prolog.assertz('sameColor(Movie1, Movie2) :- color(Movie1, Color1), color(Movie2, Color2), Color1 = Color2, Movie1 \= Movie2')
prolog.assertz('sameLanguage(Movie1, Movie2) :- language(Movie1, Language), language(Movie2, Language), Movie1 \= Movie2')
prolog.assertz('somewhatSameActors(Movie1, Movie2) :- cast(Movie1, Actors1), cast(Movie2, Actors2), intersection(Actors1, Actors2, Common), length(Common, 2)')
prolog.assertz('somewhatSameActors(Movie1, Movie2) :- cast(Movie1, Actors1), cast(Movie2, Actors2), intersection(Actors1, Actors2, Common), length(Common, 2)')
prolog.assertz('somewhatSameActors(Movie1, Movie2) :- cast(Movie1, Actors1), cast(Movie2, Actors2), intersection(Actors1, Actors2, Common), length(Common, 2)')


genre('My Date with Drew','Documentary').


//κρασάρει μετά από ένα σημείο-> θεωρω παίρνει κάθε genre ξεχωριστά και το συγκρίνει ξανα με όλα.

π.χ Avatar: Action|Fantasy|Adventure. Για κάθε κατηγορία παίρνει και εκτυπώνει ό,τι βρίσκει με κοινό ένα είδος κάθε φορά γιατί ούτως ή άλλως είναι οργανωμένο σε "2άδες" --> Πρέπει σαν λειτουργία να το βάζω έγω να ψάχνει μόνο το Action κάθε φορά

Επιθυμητή λειτουργία: Να μου δείχνει ΜΟΝΟ τις ταινίες που θα έχει 2+ κοινά genres.



In [ ]:
from pyswip import Prolog

prolog = Prolog()

q = prolog.query("similargenre('Avatar', Genre), genre(Movie, Genre)")
for result in q:
    print(result['Movie'])
q.close()



Streaming output truncated to the last 5000 lines.
Reign of Fire
Renaissance
Reno 911!: Miami
Repo Men
Resident Evil
Resident Evil: Afterlife
Resident Evil: Apocalypse
Resident Evil: Extinction
Resident Evil: Retribution
Return of the Jedi
Riddick
Ride Along 2
Ride Along
Righteous Kill
Rise of the Planet of the Apes
Risen
Road House
Robin Hood
Robin and Marian
RoboCop 3
RoboCop
RockNRolla
Rockaway
Rollerball
Romeo Is Bleeding
Romeo Must Die
Ronin
Royal Kill
Rumble in the Bronx
Run All Night
Run Lola Run
Running Scared
Rush Hour 2
Rush Hour 3
Rush
S.W.A.T.
Sabotage
Safe House
Safe
Sahara
Saints and Soldiers
Salt
Salvation Boulevard
San Andreas
Sanctum
Sands of Iwo Jima
Saving Private Perez
Scarface
Scott Pilgrim vs. the World
Screwed
See Spot Run
September Dawn
Serenity
Set It Off
Seven Samurai
Shade
Shadow Conspiracy
Shaft
Shanghai Knights
Shanghai Noon
Shaolin Soccer
Shark Tale
Sheena
Sherlock Holmes
Sherlock Holmes: A Game of Shadows
Shin Godzilla
Shinjuku Incident
Shooter
Showdown i

KeyboardInterrupt: 

Μόνο το Action κοινό:

In [ ]:
from pyswip import Prolog

prolog = Prolog()

# Καλούμε τον Prolog για να βρούμε τις ταινίες με κοινό είδος το "Action"
q = prolog.query("similargenre('Avatar', Genre), genre(Movie, 'Action')")
for result in q:
    print(result['Movie'])
q.close()


Streaming output truncated to the last 5000 lines.
Alex Cross
Alexander
Alien Zone
Alien
Alien: Resurrection
Aliens vs Predator: Requiem
Aliens
Alien³
Alive
All About the Benjamins
All Hat
All Is Lost
All The Queens Men
Alone in the Dark
Along Came a Spider
American Heist
American Hero
American Ninja 2: The Confrontation
American Outlaws
American Sniper
Amidst the Devils Wings
Anacondas: The Hunt for the Blood Orchid
Annie Get Your Gun
Ant-Man
Antitrust
Apocalypto
Armageddon
Armored
Arn: The Knight Templar
Around the World in 80 Days
Assassins
Astro Boy
Attack the Block
Avatar
Avengers: Age of Ultron
Baahubali: The Beginning
Babylon A.D.
Babys Day Out
Bad Boys II
Bad Boys
Bad Company
Bait
Ballistic: Ecks vs. Sever
Bandidas
Bandits
Bangkok Dangerous
Basic
Batman & Robin
Batman Begins
Batman Forever
Batman Returns
Batman v Superman: Dawn of Justice
Batman
Batman: The Dark Knight Returns, Part 2
Battle for the Planet of the Apes
Battle: Los Angeles
Battlefield Earth
Battleship
Beastmaster